In [9]:
import zipfile
import numpy as np
import pandas
import os
import io
from pprint import pprint
import json

In [10]:
os.chdir('retrosheet/')
os.getcwd()

decades = np.linspace(1920,2010,10)
zipfiles = []
for x in decades:
    zipfiles.append('{:.0f}'.format(x)+'seve.zip')
for filename in zipfiles:
    zip_ref = zipfile.ZipFile(filename, 'r')
    zip_ref.extractall('.')
    zip_ref.close()
print('Done')

Done


In [16]:

os.chdir('sportsball/')
os.getcwd()

'/home/idies/workspace/persistent/sportsball'

In [17]:
#os.getcwd()
os.chdir('retrosheet/')
yeararray = np.linspace(1921,2016,96)
teamfilearray = []
for x in yeararray:
#    print(x)
    teamfilearray.append('TEAM{:.0f}'.format(x))
#teamfilearray
teams = np.empty((0,4),dtype=object)
actualfiles = os.listdir()
for y in teamfilearray:
    if (os.path.isfile(y)):
        for line in open(y).readlines():
            thisteam = []
            for z in line.split(','):
                thisteam.append(z)
            thisteam[-1] = thisteam[-1][0:-1]  # clear newline character at end
            if (thisteam[0] not in teams[:,0]):
                teams = np.vstack((teams,thisteam))
os.chdir('../')
#os.getcwd()
teams_df = pandas.DataFrame(teams)
teams_df.columns = ['teamID','leagueID','city','nickname']
teams_df = teams_df.set_index('teamID')
teams_df.to_csv('teams.csv')
print('Done')
#teams_df.shape
#with open('teams.csv','w') as f:
#f.write(teams)

Done


In [18]:
allfiles = []

yeararray = np.linspace(1921,2016,96)

#teamarray = np.empty((0,4),dtype=object)
for oneyear in yeararray:
    for oneteam in teams_df.index:
        afile = 'retrosheet/' + '{:.0f}'.format(oneyear)+str(oneteam)+'.EV'+str(teams_df.loc[oneteam]['leagueID'])
        if (os.path.isfile(afile)):
            allfiles.append(afile)
#print(allfiles)
print('files identified')


files identified


In [ ]:
allgames = []
gamecount = 0
#allfiles=['retrosheet/raw/1962BOS.EVA']
for filename in allfiles:
    for line in io.open(filename,encoding='utf-8').readlines():
        linearray = line.split(',')
        linearray[-1] = linearray[-1][0:-1]  # cut off newline character
        if (linearray[0] == 'id'):
            # 'id' starts the next game, so immediately advance gamecount  

            # record prior game data by adding it to the gamelogs list, unless gamecount = -1
            if (gamecount > 0):
                thisgamedict['info'] = info_dict
                thisgamedict['visiting_starters'] = vis_start_list.tolist()
                thisgamedict['home_starters'] = home_start_list.tolist()

                # record game subs only if there were any subs
#                if (len(vis_sub_list) > 0):
#                    thisgamedict['visiting_subs'] = vis_sub_list.tolist()
#                if (len(home_sub_list) > 0):
#                    thisgamedict['home_subs'] = home_sub_list.tolist()

                thisgamedict['events'] = events.tolist()
                thisgamedict['data'] = data_list.tolist()
                # append the dictionary describing this game to the list of games
                allgames.append(thisgamedict)

            # advance the gamecount and initialize a new game
            gamecount = gamecount + 1
            thisgamedict = {}
            info_dict = {}
            start_dict = {}
            vis_start_list = np.empty((9,4),dtype=object)
            home_start_list = np.empty((9,4),dtype=object)
            vis_sub_list = np.empty((0,4),dtype=object)
            home_sub_list = np.empty((0,4),dtype=object)
            events = np.empty((0,11),dtype=object)
            data_list = np.empty((0,2),dtype=object)
            batterscount = 0
            eventnum = 0

            # the first thing to go into the new game is the game id.
            # the [0:-1] is there to cut off the newline character
            thisgamedict['id'] = linearray[1]

        elif (linearray[0] == 'version'):
            thisgamedict['version'] = linearray[1]
        elif (linearray[0] == 'info'):
            info_dict[linearray[1]] = linearray[2]
        elif (linearray[0] == 'start'):
            batterscount = batterscount + 1
            if (linearray[3] == '0'):
                vis_start_list[batterscount-1,0] = linearray[1]
                vis_start_list[batterscount-1,1] = linearray[2]
                vis_start_list[batterscount-1,2] = linearray[4]
                vis_start_list[batterscount-1,3] = linearray[5]
            elif (linearray[3] == '1'):
                home_start_list[batterscount-10,0] = linearray[1]
                home_start_list[batterscount-10,1] = linearray[2]
                home_start_list[batterscount-10,2] = linearray[4]
                home_start_list[batterscount-10,3] = linearray[5]        
            else:
                print('NEITHER VISITOR NOR HOME????')
                
        elif (linearray[0] == 'data'):
            thisdata = np.empty((2),dtype=object)
            thisdata[0] = linearray[2]
            thisdata[1] = linearray[3]
            data_list = np.vstack((data_list,thisdata))
        else:
#            print(linearray)
            thisevent = []
            thisevent.append(eventnum)
            thisevent.append(linearray[0])  # say what type of event: play/com/sub/etc.
            if (linearray[0] == 'play'):
                for i in range(1,7):
                    thisevent.append(linearray[i])
                for i in range(8,11):   # comment, sub_batting_order, sub_position empty
                    thisevent.append('')
            elif (linearray[0] == 'com'):
                for i in range(1,7):
                    thisevent.append('')
                thisevent.append(linearray[1])
                for i in range(9,11):
                    thisevent.append('')
            elif (linearray[0] == 'sub'):
                thisevent.append('')    # inning, fill in later
                thisevent.append(linearray[3])  # visitor/home
                thisevent.append(linearray[1])  # playerID of player coming in
                for i in range(4,8):
                    thisevent.append('')  # count/pitches/result/comment all blank
                thisevent.append(linearray[4]) # sub's batting order
                thisevent.append(linearray[5]) # sub's fielding position
            elif (linearray[0] == 'badj'):
                thisevent.append('')    # inning, fill in later
                thisevent.append('')  # visitor/home, fill in later
                for i in range(4,8):
                    thisevent.append('')  # count/pitches/result/comment all blank                
                thisevent.append(linearray[1])  # playerID of adjusting batter
                thisevent.append(linearray[2]) # batter hand adjusted to
                thisevent.append('')  # add blank                
            elif (linearray[0] == 'padj'):
                thisevent.append('')    # inning, fill in later
                thisevent.append('')  # visitor/home, fill in later
                thisevent.append(linearray[1])  # playerID of adjusting pitcher
                thisevent.append('')  # add blank                                
                thisevent.append(linearray[2]) # pitcher hand adjusted to
            elif (linearray[0] == 'ladj'):
                thisevent.append('')    # inning, fill in later
                thisevent.append(linearray[1])  # visitor/home
                for i in range(3,8):
                    thisevent.append('')  # playerID/count/pitches/result/comment all blank
                thisevent.append(linearray[2]) # new batter
                thisevent.append('') # add blank  
            else:
                print(linearray)
            events = np.vstack((events,thisevent))
            eventnum = eventnum + 1

#for i in range(0,len(allgames)):
#    print('i = ',i)
#    pprint(allgames[i])
#    print('\n')
print('ahoy')
#pprint(allgames)

In [8]:
pprint(allgames[-1]['id'])
print(len(allgames))

'SDN196909230'
51100


In [ ]:
with open('0events.json', 'w') as f:
    json.dump(allgames, f)
print('ok')

In [ ]:
with open('0events.json','r') as f:
    allgames = json.load(f)
allgames

In [ ]:
allgames[0]['events']

In [ ]:
nEvents = 0
for game in allgames:
    nEvents = nEvents + len(game['events'])
print('{0:,.0f} events found'.format(nEvents))
events_array = np.empty((nEvents,3),dtype=object)
for game in allgames:
    for j in range(0,len(game['events'])):
        print(game['events'])
#        events_array[eventno,0] = game['events'][j][0]
#        events[eventno,1] = game['info']['visteam']
#        events[eventno,2] = game['info']['hometeam']
events_array